In [11]:
# univariate lstm example
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import ConvLSTM2D
from tensorflow.keras import optimizers

In [12]:
df = pd.read_csv("pc_2038.csv")

In [13]:
df = df['Product_Demand']

In [14]:
df

0      20
1       1
2       2
3       2
4       1
       ..
725     1
726     1
727    20
728     1
729     1
Name: Product_Demand, Length: 730, dtype: int64

In [15]:
# preparing independent and dependent features
def prepare_data(timeseries_data, n_features):
	X, y =[],[]
	for i in range(len(timeseries_data)):
		# find the end of this pattern
		end_ix = i + n_features
		# check if we are beyond the sequence
		if end_ix > len(timeseries_data)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = timeseries_data[i:end_ix], timeseries_data[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [16]:
# define input sequence
timeseries_data = df.to_numpy()
# choose a number of time steps
n_steps = 100
# split into samples
X, y = prepare_data(timeseries_data, n_steps)

In [17]:
print(X),print(y)

[[20  1  2 ...  5  3  1]
 [ 1  2  2 ...  3  1  1]
 [ 2  2  1 ...  1  1  2]
 ...
 [20  1  1 ... 20  1  1]
 [ 1  1  1 ...  1  1 20]
 [ 1  1  3 ...  1 20  1]]
[ 1  2  1  1  4  1  1  1 20  1  2  1  1  1  2  1  1  1  5  1  2  1  3  1
  1  1  1  1  1  1  1  1  3  3  1  1  2  1  1  1  1  1  1  1  2  2  3  1
  2  1  1  1  1  1  2  1  1  1  1  1  1  1  8 20  1  1  1  1  1  1  3  8
  1  1  1  2  1  2  1  1  1  1  1  1  2  3  3 20 24  1  1  1  1  3  2  1
  1  2  2  1  1  1  6 14  1  1  1  3  2  2  3  1  1  2  1  1  1  2  1  1
  1  1  3  1  1  1 20  1  1  1  2  1  1  1 20  1  1  1  1  1  1  1  1  1
  1 20  1  1  1  1  1  8  3  1  1  1 19 30  1  1  1  1  1  1  1  2  1  1
 20  1  3  1  1  1  1  1  3  1  1  1  2  1  1  1  2  1  1  1  1  1 20  2
  1  1  1  1  1  5  1  1  2  2  1  1  1  1  3  1  1 20  1  1  1  1  1  1
  1  1  1  1  1  1  2  2  1  1  2  1  1  1 20  3  1  1  1  1 20  1  1  1
  1  3  4  1  1  1  1  1  2  6 20  1 20  1  2  1  1  1  6 14  1  1  1  1
  2  1  1  2  3  1  1  1  2  1  1  1  1 2

(None, None)

In [18]:
X.shape

(630, 100)

In [19]:
# choose a number of time steps
n_steps = 100
# reshape from [samples, timesteps] into [samples, timesteps, rows, columns, features]
n_features = 1
n_seq = 1
n_steps = 100
X = X.reshape((X.shape[0], n_seq, 1, n_steps, n_features))

In [20]:
# define model
model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(n_seq, 1, n_steps, n_features)))
model.add(Flatten())
model.add(Dense(1, activation='relu'))
adam = optimizers.Adam(lr=0.001)
model.compile(optimizer=adam, loss='mse')

In [21]:
# fit model
model.fit(X, y, epochs=150, verbose=1)

Train on 630 samples
Epoch 1/150
630/630 [==============================] - 3s 5ms/sample - loss: 36.9904
Epoch 2/150
630/630 [==============================] - 1s 1ms/sample - loss: 34.6046
Epoch 3/150
630/630 [==============================] - 1s 1ms/sample - loss: 33.4892
Epoch 4/150
630/630 [==============================] - 1s 1ms/sample - loss: 32.4433
Epoch 5/150
630/630 [==============================] - 1s 1ms/sample - loss: 31.2350
Epoch 6/150
630/630 [==============================] - 1s 1ms/sample - loss: 30.3620
Epoch 7/150
630/630 [==============================] - 1s 1ms/sample - loss: 29.1713
Epoch 8/150
630/630 [==============================] - 1s 1ms/sample - loss: 28.7148
Epoch 9/150
630/630 [==============================] - 1s 1ms/sample - loss: 28.1320
Epoch 10/150
630/630 [==============================] - 1s 1ms/sample - loss: 27.1161
Epoch 11/150
630/630 [==============================] - 1s 1ms/sample - loss: 26.1470
Epoch 12/150
630/630 [====================

In [22]:
model.save("model2038.h5")